## Setup

Ollama is a library for running open source LLMs locally.

If you haven't already, download Ollama at [here](https://ollama.com/)

then, open your favorite terminal and run

`ollama run llama3`

Next, lets install our python dependencies, including langchain, faiss, ollama, and firecrawl-py

In [2]:
%pip install --upgrade --quiet groq firecrawl-py
%pip install --force-reinstall typing-extensions==4.5
%pip install --force-reinstall groq==0.5.0

Note: you may need to restart the kernel to use updated packages.
  Obtaining dependency information for typing-extensions==4.5 from https://files.pythonhosted.org/packages/31/25/5abcd82372d3d4a3932e1fa8c3dbf9efac10cc7c0d16e78467460571b404/typing_extensions-4.5.0-py3-none-any.whl.metadata
  Using cached typing_extensions-4.5.0-py3-none-any.whl.metadata (8.5 kB)
Using cached typing_extensions-4.5.0-py3-none-any.whl (27 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.11.0
    Uninstalling typing_extensions-4.11.0:
      Successfully uninstalled typing_extensions-4.11.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastapi 0.110.0 requires typing-extensions>=4.8.0, but you have typing-extensions 4.5.0 which is incompatible.
chromadb 0.4.24 requires tenacity>=8.2.3, but you have tenacity 8.2.2 which is incompatib

# Load website with Firecrawl

In [3]:
from firecrawl import FirecrawlApp  # Importing the FireCrawlLoader

url = "https://about.fb.com/news/2024/04/introducing-our-open-mixed-reality-ecosystem/"
firecrawl = FirecrawlApp(
    api_key="fc-",
)
page_content = firecrawl.scrape_url(url=url,  # Target URL to crawl
    params={
        "pageOptions":{
            "onlyMainContent": True # Ignore navs, footers, etc.
        }
    })
print(page_content)

{'content': '\n\n[Back to Newsroom](https://about.fb.com/news)\n\nMeta\n\nApril 22, 2024April 22, 2024\n\n![Image showing a box labeled "Built with Meta Horizon OS"](https://about.fb.com/wp-content/uploads/2024/04/RL-OEM_Ecosystem-Announcement_Header-1.jpg)\n\nTakeaways\n---------\n\n*   We’re opening up the operating system that powers our Meta Quest devices to third-party hardware makers.\n*   We’re working with leading technology companies to create a new ecosystem of mixed reality devices, and we’re making it even easier for developers to build mixed reality apps that reach a wide audience.\xa0\n*   This more open ecosystem will help bring the power of mixed reality to more people and businesses around the world.\n\nToday, we’re taking a major step toward our vision for a more open computing platform for the metaverse. We’re opening up the operating system that powers our Meta Quest devices to third-party hardware makers, giving developers a larger ecosystem to build for and ultima

## Retrieval and Generation

In [4]:
from groq import Groq

client = Groq(
    api_key="gsk_",  # Note: Replace 'API_KEY' with your actual Groq API key
)

extract = ["summary","title of the article","date","companies_building_quest"]
completion = client.chat.completions.create(
    model="llama3-8b-8192",
    messages=[
        {
            "role": "system",
            "content": "You are a legal advisor who extracts information from documents in JSON."
        },
        
        {
            "role": "user",
            "content": f"Extract the following information from the provided documentation:\Page content:\n\n{page_content}\n\nInformation to extract: {extract}"
        }
    ],
    temperature=0,
    max_tokens=1024,
    top_p=1,
    stream=False,
    stop=None,
    response_format={"type": "json_object"}
)

print(completion.choices[0].message)
import json

# Pretty print the JSON response
print(json.dumps(str(completion.choices[0].message.content), indent=4))



ChoiceMessage(content='{\n"summary": "Meta is opening up the operating system that powers its Meta Quest devices to third-party hardware makers and making it easier for developers to build mixed reality apps that reach a wide audience.",\n"title": "Introducing Our Open Mixed Reality Ecosystem | Meta",\n"date": "April 22, 2024",\n"companies_building_quest": ["ASUS\'s Republic of Gamers", "Lenovo", "Xbox"],\n"people_testimonials": []\n}', role='assistant', tool_calls=None)
"{\n\"summary\": \"Meta is opening up the operating system that powers its Meta Quest devices to third-party hardware makers and making it easier for developers to build mixed reality apps that reach a wide audience.\",\n\"title\": \"Introducing Our Open Mixed Reality Ecosystem | Meta\",\n\"date\": \"April 22, 2024\",\n\"companies_building_quest\": [\"ASUS's Republic of Gamers\", \"Lenovo\", \"Xbox\"],\n\"people_testimonials\": []\n}"
